In [2]:
import typing
import os

import pandas
import datasets
import trl
import peft

import cltrier_lib

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
MODEL_SLUG: str = "meta-llama/Meta-Llama-3-8B-instruct"
RAW_DATASET: str = "../data/interim/twitter.german.dataset.preds.csv"

In [5]:
dataset: typing.List[typing.Dict] = [
    cltrier_lib.inference.schemas.Chat(messages=[
        cltrier_lib.inference.schemas.Message(role="system", content=f"You are a social media user with a political {row['leaning']} leaning. Respond to the following Tweet:"),
        cltrier_lib.inference.schemas.Message(role="user", content=row["text_post"]),
        cltrier_lib.inference.schemas.Message(role="assistant", content=row["text_reply"])
    ]).model_dump()
    for _, row in pandas.read_csv(RAW_DATASET, index_col=0).iterrows()
]
dataset[:3]

[{'messages': [{'role': 'system',
    'content': 'You are a social media user with a political neutral leaning. Respond to the following Tweet:'},
   {'role': 'user',
    'content': 'Nicht der #Verbrenner schadet dem #Klima, sondern der fossile Sprit, mit dem er fährt. Wir haben diese Woche den Weg für klimaneutrale #eFuels freigemacht. Damit könnten die mehr als 45 Mio. Diesel- und Benzin-Fahrzeuge auf unseren Straßen in Zukunft klimaneutral unterwegs sein.'},
   {'role': 'assistant',
    'content': '@christianduerr Sie haben wirklich keine Ahnung.'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a social media user with a political neutral leaning. Respond to the following Tweet:'},
   {'role': 'user',
    'content': 'Wo waren die ganzen plötzlichen #Kernkraftbefürworter in #Altparteien, Verbänden &amp; Medien in den letzten Jahren? Warum stimmte die #umfaller: #fdp bis zuletzt im Bundestag gegen Laufzeitverlängerungen? Fakt ist: nur die #AfD lag von Anfang an &amp; jah

In [6]:
sft_config = trl.SFTConfig(
    num_train_epochs=10,
    packing=True, 
    output_dir="./sft_results",
    logging_steps=50
)

peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
trainer = trl.SFTTrainer(
    MODEL_SLUG,
    args=sft_config,
    train_dataset=datasets.Dataset.from_pandas(pandas.DataFrame(data=dataset)),
    peft_config=peft_config,
)

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
100,3.132800
200,2.979400
